## Routes of CO2 emissions

In [ ]:
# Some amazing visualization libraries used for data 
import folium
from geopy.geocoders import Nominatim
from folium.plugins import MarkerCluster
import plotly.cexpress as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

### Using average co2 emission per operator

In [377]:
# Helper to load opensky dataset
def load_dataset_opensky(filename, sep=','):
    dirpath = os.path.join(".","data", "opensky-network")
    filepath = os.path.join(dirpath, filename)
    raw_data = pd.read_csv(filepath, sep, header=None, encoding='utf-8', low_memory=False)
    return raw_data
# Helper to load openflight datasets
def load_dataset_openflight(filename, sep=','):
    dirpath = os.path.join(".","data", "openflight")
    filepath = os.path.join(dirpath, filename)
    raw_data = pd.read_csv(filepath, sep, header=None, encoding='utf-8')
    return raw_data
# Helper to clearn the data in openflight datasets
def remove_backslash_n(dataset):
    return dataset.replace('\\N', np.nan)

from math import sin, cos, sqrt, atan2, radians
#Ref: https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
def distance(lat1, lon1, lat2, lon2):
    """ Return the distance between two lat/long coordinates"""
    # approximate radius of earth in km
    R = 6373.0
    # convert to radian and take abs value
    lat1 = radians(abs(lat1))
    lon1 = radians(abs(lon1))
    lat2 = radians(abs(lat2))
    lon2 = radians(abs(lon2))
    
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Helpers to compute add columns
def distance_from_row(row):
    return distance(row['lat_dep'], row['long_dep'], row['lat_dest'], row['long_dest'])
def co2_total(row):
    return row['distance']/100*row['CO2 kg/km']
def co2_total_per_seat(row):
    return row['distance']/100*row['CO2 per seat kg/100km']

# Helper to get a range of colors
from colour import Color
from math import floor
green = Color("green")

colors = list(green.range_to(Color("red"),50))
def get_color(v):
    return colors[floor(v*len(colors)-0.0001)].get_hex()
green.get_hex()

# Helper to plot interactive scattergeo plot
def plot_scatterplot(dataset, feature='CO2 per seat kg/100km', title='Insert a title please !', europe=False):
    """Plot a dataset of routes on a map"""
    route_plot = go.Figure()
    # We reset index in order to go through it below
    dataset = dataset.reset_index()
    for i in range(len(dataset)):
        route_plot.add_trace(
            go.Scattergeo(
                lon = [dataset['long_dep'][i], dataset['long_dest'][i]],
                lat = [dataset['lat_dep'][i], dataset['lat_dest'][i]],
                mode = 'lines',
                line = dict(
                    width = 1,
                    color = get_color((dataset['CO2 per seat kg/100km'][i]-dataset['CO2 per seat kg/100km'].min())\
                                      /(dataset['CO2 per seat kg/100km'].max()-dataset['CO2 per seat kg/100km'].min()))
                ),   
                opacity = 1.0,
                #hoverinfo = 'text',
                #text = co2_dataset['operator'][i] + ' : ' + co2_dataset['departure'][i] +  ' - ' + co2_dataset['destination'][i],
            )
        )
    if (not(europe)):
        route_plot.update_layout(
            title_text = title,
            showlegend = False,
            geo = go.layout.Geo(
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )
    else:
        route_plot.update_layout(
            title_text = title,
            showlegend = False,
            geo = go.layout.Geo(
                scope='europe',
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )
    
    
            

    route_plot.show()


# Define the list of european countries
eu_countries = ['Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus',\
                'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',\
                'Germany', 'Greece', 'Hungary', 'Ireland', 'Italy', 'Latvia',\
                'Luxembourg', 'Lithuania', 'Malta', 'Netherlands', 'Poland',\
                'Portugal', 'Romania', 'Slovakia', 'Slovenia', 'Spain',\
                'Sweden', 'United Kingdom']

In [363]:
# Load the opensky datasets
aircrafts_raw = load_dataset_opensky('aircraftDatabase.csv.xz')
flights_records_raw = load_dataset_opensky('log.csv.xz')

# Name the columns
aircrafts_raw.columns = ["icao24","registration","manufacturericao","manufacturername","model","typecode","serialnumber","linenumber","icaoaircrafttype","operator","operatorcallsign","operatoricao","operatoriata","owner","testreg","registered","reguntil","status","built","firstflightdate","seatconfiguration","engines","modes","adsb","acars","notes","categoryDescription"]
flights_records_raw.columns = ['icao24', 'duration', 'callsign', 'departure', 'destination']

# Remove first row because it didn't load properly
aircrafts_raw = aircrafts_raw.drop(0, axis=0)
flights_records_raw = flights_records_raw.drop(0, axis=0)

# Join aircrafts_raw and fligths_records_raw to get a dataset with operator, model, departure and destination
dataset_raw = aircrafts_raw.join(flights_records_raw.set_index('icao24'), on='icao24')\
[['operator','model','departure', 'destination']]

# Drop the nan values
dataset_raw = dataset_raw.dropna()
dataset_raw.head()

,operator,model,departure,destination
45,Indian Air Force,C-17A Globemaster III,VEGK,VIDX
45,Indian Air Force,C-17A Globemaster III,VIDP,VEGK
45,Indian Air Force,C-17A Globemaster III,VIDX,VOYK
45,Indian Air Force,C-17A Globemaster III,VOMM,VIDX
45,Indian Air Force,C-17A Globemaster III,VIDX,VOMM


In [364]:
# Load the openflight datasets
airports_raw = load_dataset_openflight('airports.dat.xz')
airports_raw.columns = ['Airport_ID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 'Altitude', 'Timezone', 'DST', 'Tz_database', 'Type', 'Source']
airports  = remove_backslash_n(airports_raw)
location_airports_departure = airports[['Name', 'ICAO', 'Latitude', 'Longitude']]
location_airports_destination = location_airports_departure.copy()
location_airports_departure.columns = ['name_dep', 'icao', 'lat_dep', 'long_dep']
location_airports_destination.columns = ['name_dest', 'icao', 'lat_dest', 'long_dest']
airports_country = airports[['ICAO', 'Country']]
airports_country.columns = ['icao', 'country']
airports_country = airports_country.drop_duplicates()

# Retrieve the EU airports
eu_airports_country = airports_country[airports_country.country.isin(eu_countries)]
# print(eu_airports_country['country'].sort_values().drop_duplicates().count())

# Retrieve the Swiss airports
swiss_airports = airports_country[airports_country.country.isin(['Switzerland'])]
swiss_airports = swiss_airports.drop_duplicates()

# Create a dataset with flight routes
merged_dep = pd.merge(left=dataset_raw,right=location_airports_departure, left_on='departure', right_on='icao')
merged_route = pd.merge(left=merged_dep,right=location_airports_destination, left_on='destination', right_on='icao')
merged_route.head()

,operator,model,departure,destination,name_dep,icao_x,lat_dep,long_dep,name_dest,icao_y,lat_dest,long_dest
0,Air India Regional,ATR 72 600,VEGK,VIDP,Gorakhpur Airport,VEGK,26.7397,83.449699,Indira Gandhi International Airport,VIDP,28.5665,77.103104
1,Air India Regional,ATR 72 600,VEGK,VIDP,Gorakhpur Airport,VEGK,26.7397,83.449699,Indira Gandhi International Airport,VIDP,28.5665,77.103104
2,Air India Regional,ATR 72 600,VEGK,VIDP,Gorakhpur Airport,VEGK,26.7397,83.449699,Indira Gandhi International Airport,VIDP,28.5665,77.103104
3,Air India Regional,ATR 72 600,VEGK,VIDP,Gorakhpur Airport,VEGK,26.7397,83.449699,Indira Gandhi International Airport,VIDP,28.5665,77.103104
4,Air India Regional,ATR 72 600,VEGK,VIDP,Gorakhpur Airport,VEGK,26.7397,83.449699,Indira Gandhi International Airport,VIDP,28.5665,77.103104


In [365]:
# Get a smaller dataset with flight routes, model is not interesting because we'll consider average co2 emissions/companies
# later on
clean_routes = merged_route[['operator', 'departure', 'destination', 'name_dep',
       'lat_dep', 'long_dep', 'name_dest', 'lat_dest', 'long_dest']]

# Retrieve co2 emissions information for each companies
co2_airlines_dataset = plot_airlines[['Operator', 'CO2 per seat kg/100km', 'CO2 kg/km']]

# Merge the two previous datasets
co2_dataset = pd.merge(left=clean_routes,right=co2_airlines_dataset, left_on='operator', right_on='Operator')
co2_dataset = co2_dataset.drop_duplicates()

# Add columns to the dataset
co2_dataset['distance'] = co2_dataset[['lat_dep', 'long_dep', 'lat_dest', 'long_dest']].apply(distance_from_row, axis=1)
co2_dataset['CO2 kg'] = co2_dataset[['distance', 'CO2 kg/km']].apply(co2_total, axis=1)
co2_dataset['CO2 per seat kg'] = co2_dataset[['distance', 'CO2 per seat kg/100km']].apply(co2_total_per_seat, axis=1)

# Clean the dataset
co2_dataset = co2_dataset[co2_dataset.distance != 0]
co2_dataset = co2_dataset.reset_index().drop('index', axis=1)

# Get the less polluting company for each route
idx = co2_dataset.groupby(['departure', 'destination'])['CO2 per seat kg/100km'].transform(min) == co2_dataset['CO2 per seat kg/100km']
co2_dataset_bestcompany = co2_dataset[idx]
co2_dataset_bestcompany.head()

,operator,departure,destination,name_dep,lat_dep,long_dep,name_dest,lat_dest,long_dest,Operator,CO2 per seat kg/100km,CO2 kg/km,distance,CO2 kg,CO2 per seat kg
0,Ethiopian Airlines,OLBA,VIDP,Beirut Rafic Hariri International Airport,33.820900,35.488400,Indira Gandhi International Airport,28.5665,77.103104,Ethiopian Airlines,6.106321,14.275292,3975.573512,567.524732,242.761295
1,Ethiopian Airlines,LEMD,VIDP,Adolfo Suárez Madrid–Barajas Airport,40.471926,-3.562640,Indira Gandhi International Airport,28.5665,77.103104,Ethiopian Airlines,6.106321,14.275292,6676.170542,953.042846,407.668428
2,Ethiopian Airlines,LLBG,VIDP,Ben Gurion International Airport,32.011398,34.886700,Indira Gandhi International Airport,28.5665,77.103104,Ethiopian Airlines,6.106321,14.275292,4047.381070,577.775470,247.146095
3,Ethiopian Airlines,OERK,VIDP,King Khaled International Airport,24.957600,46.698799,Indira Gandhi International Airport,28.5665,77.103104,Ethiopian Airlines,6.106321,14.275292,3038.021244,433.686407,185.511340
4,Ethiopian Airlines,ZUCK,VIDP,Chongqing Jiangbei International Airport,29.719200,106.641998,Indira Gandhi International Airport,28.5665,77.103104,Ethiopian Airlines,6.106321,14.275292,2864.765565,408.953653,174.931792


In [368]:
# Make a dataset with the routes starting from Switzerland
co2_dataset_bestcompany_from_switzerland = pd.merge(left=co2_dataset_bestcompany,right=swiss_airports, left_on='departure', right_on='icao')

# Plot the routes starting from Switzerland 
plot_scatterplot(co2_dataset_bestcompany_from_switzerland, feature='CO2 per seat kg/100km', title='Flight routes starting from Swiss airports')

*We observe that there is a lack of contrast due to outlier companies that have very high CO2 emissions per seat/100km, we have to delete them to have a better view.*

In [370]:
plot_scatterplot(co2_dataset_bestcompany_from_switzerland.sort_values(by='CO2 per seat kg/100km', ascending=False)[2:],\
                 feature='CO2 per seat kg/100km', title='Flight routes starting from Swiss airports without outliers')

In [380]:
print('max: {} || min: {}'.format(co2_dataset_bestcompany_from_switzerland['CO2 per seat kg/100km'].max(), co2_dataset_bestcompany_from_switzerland['CO2 per seat kg/100km'].min()))

max: 8.044998394186043 || min: 5.525246744181813


*We now plot what are the less polluting by km routes from Switzerland to EU countries and we directly remove the outliers which remain the same.*

In [378]:
co2_dataset_bestcompany_model_from_switzerland_to_europe = pd.merge(left=co2_dataset_bestcompany_model_from_switzerland,right=eu_airports_country, left_on='destination', right_on='icao')
plot_scatterplot(co2_dataset_bestcompany_model_from_switzerland_to_europe.sort_values(by='CO2 per seat kg/100km', ascending=False)[2:],\
                 feature='CO2 per seat kg/100km', title='Flight routes starting from Swiss airports to EU countries without outliers', europe=True)


*We clearly saw that there are better destinations to go for if you care for the environment even for the similar distances. In fact, we see a broad range of colors for distance which are similar.*